# ML Quality Eval: Validate on Test Set
Uses YOUR acceleration calculation: predicted_speed(t) - speed_prev1(t)

In [ ]:
# CELL 1: Parameters
RUN_TIMESTAMP = "2025-01-01_00-00-00"
INPUT_TEST_DATA = "s3://models-quality-eval-ml/test/test_data.pkl"
INPUT_ML_MODEL_PATH = "s3://models-quality-eval-ml/models/speed_accel_model.pkl"
OUTPUT_METRICS_PATH = "s3://models-quality-eval-ml/metrics/quality_metrics.json"
OUTPUT_PLOT_PATH = "s3://models-quality-eval-ml/metrics/validation_plots.png"

# Quality Thresholds (relaxed for single-file testing)
MIN_R2_SCORE = 0.85
MAX_SPEED_RMSE = 2.5  # m/s
MAX_ACCEL_RMSE = 0.7  # m/s²
MAX_SPEED_MAE = 2.0   # m/s
MAX_ACCEL_MAE = 0.5   # m/s²

MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"

In [ ]:
# CELL 2: Imports
import pandas as pd
import numpy as np
import pickle
import json
import io
import s3fs
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

sns.set_style('whitegrid')
print("✅ Libraries imported successfully!")

In [ ]:
# CELL 3: MinIO Configuration
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)

storage_options = {
    "key": MINIO_ACCESS_KEY,
    "secret": MINIO_SECRET_KEY,
    "client_kwargs": {"endpoint_url": MINIO_ENDPOINT}
}

In [ ]:
# CELL 4: Load Test Data and Model
print(f"=== ML Quality Validation ===")
print(f"Run Timestamp: {RUN_TIMESTAMP}")
print(f"\nLoading artifacts...")

# Load test data
try:
    with fs.open(INPUT_TEST_DATA, 'rb') as f:
        df_test = pickle.load(f)
    
    if isinstance(df_test, pd.DataFrame):
        print(f"✅ Loaded test DataFrame with {len(df_test):,} rows")
    else:
        raise TypeError(f"Expected DataFrame, got {type(df_test)}")
    
except FileNotFoundError:
    print(f"❌ Error: {INPUT_TEST_DATA} not found. Run step 01 first.")
    raise

# Load trained model
try:
    with fs.open(INPUT_ML_MODEL_PATH, 'rb') as f:
        model_artifact = pickle.load(f)
    
    scaler = model_artifact['scaler']
    speed_model = model_artifact['speed_model']
    feature_cols = model_artifact['feature_cols']
    model_name = model_artifact.get('speed_model_name', 'Unknown')
    train_metrics = model_artifact.get('train_metrics', {})
    
    print(f"✅ Model loaded: {model_name}")
    if train_metrics:
        print(f"   Training R²: {train_metrics.get('r2', 'N/A'):.4f}")
        print(f"   Training RMSE: {train_metrics.get('rmse', 'N/A'):.4f} m/s")
except FileNotFoundError:
    print(f"❌ Error: {INPUT_ML_MODEL_PATH} not found. Run step 03 first.")
    raise

print(f"\nTest dataset shape: {df_test.shape}")

In [ ]:
# CELL 5: Feature Engineering (Same as Training) - CLEANED
print("\nPerforming feature engineering on test set...")

column_mapping = {
    'timestamp_sensor': 'timestamp',
    'latitude': 'position_lat',
    'longitude': 'position_long',
    'speed_ms': 'speed_mps',
    'altitude': 'enhanced_altitude',
    'acc_forward': 'acceleration',
}

for old, new in column_mapping.items():
    if old in df_test.columns and new not in df_test.columns:
        df_test.rename(columns={old: new}, inplace=True)


if 'trip_id' in df_test.columns:
    df_test = df_test.sort_values(['trip_id', 'seconds_elapsed'])
else:
    df_test = df_test.sort_values('seconds_elapsed')


if 'position_lat' in df_test.columns and 'position_long' in df_test.columns:
    if 'trip_id' in df_test.columns:
        df_test['delta_lat'] = df_test.groupby('trip_id')['position_lat'].diff().fillna(0)
        df_test['delta_lon'] = df_test.groupby('trip_id')['position_long'].diff().fillna(0)
    else:
        df_test['delta_lat'] = df_test['position_lat'].diff().fillna(0)
        df_test['delta_lon'] = df_test['position_long'].diff().fillna(0)
    df_test['delta_dist'] = np.sqrt(df_test['delta_lat']**2 + df_test['delta_lon']**2)
else:
    df_test['delta_lat'] = 0
    df_test['delta_lon'] = 0
    df_test['delta_dist'] = 0


if 'enhanced_altitude' in df_test.columns:
    if 'trip_id' in df_test.columns:
        df_test['elev_gain_m'] = df_test.groupby('trip_id')['enhanced_altitude'].diff().fillna(0)
    else:
        df_test['elev_gain_m'] = df_test['enhanced_altitude'].diff().fillna(0)
else:
    df_test['elev_gain_m'] = 0


if 'label_traffic' in df_test.columns:
    traffic_map = {'heavy': 2, 'moderate': 1, 'light': 0}
    df_test['traffic_level'] = df_test['label_traffic'].map(traffic_map).fillna(1)
else:
    df_test['traffic_level'] = 1


if 'bearing' not in df_test.columns:
    df_test['bearing'] = 0
    
if 'trip_id' in df_test.columns:
    df_test['heading_change'] = df_test.groupby('trip_id')['bearing'].diff().fillna(0)
else:
    df_test['heading_change'] = df_test['bearing'].diff().fillna(0)
    
df_test['turn_count'] = (np.abs(df_test['heading_change']) > 15).astype(int)


df_test = df_test.fillna(0)

print("✅ Feature engineering complete")
print("⚠️  NOTE: speed_mps_prev1/prev2 will be created in autoregressive loop (Cell 7)")

In [ ]:
# CELL 6: Prepare Test Features
# Ensure all feature columns exist
missing_cols = [col for col in feature_cols if col not in df_test.columns]
if missing_cols:
    print(f"⚠️  Warning: Missing columns {missing_cols}. Creating with zeros.")
    for col in missing_cols:
        df_test[col] = 0

X_test = df_test[feature_cols].values

# Scale features using trained scaler
X_test_scaled = scaler.transform(X_test)

print(f"\n✅ Test data prepared:")
print(f"   X_test shape: {X_test.shape}")

In [ ]:
# CELL 7: Make Autoregressive Predictions (Real-World Simulation)

print("\n=== GENERATING AUTOREGRESSIVE PREDICTIONS (High Accuracy Mode) ===")
print("⚠️ This loops through data row-by-row. It may take a few minutes for large datasets.")

# 1. Sort data is CRITICAL for autoregressive logic
if 'trip_id' in df_test.columns:
    df_test = df_test.sort_values(['trip_id', 'seconds_elapsed']).reset_index(drop=True)
else:
    df_test = df_test.sort_values('seconds_elapsed').reset_index(drop=True)

# 2. Initialize columns
df_test['predicted_speed'] = 0.0

idx_prev1 = feature_cols.index('speed_mps_prev1')
idx_prev2 = feature_cols.index('speed_mps_prev2')

print(f"Feature columns count: {len(feature_cols)}")
print(f"Updating feature indices: prev1={idx_prev1}, prev2={idx_prev2}")

# 3. Start Autoregressive Loop

predictions = []

# Iterasi baris per baris
for i in tqdm(range(len(df_test)), desc="Predicting per second"):
    
    # A. LOGIC UPDATE FEATURE DARI PREDIKSI SEBELUMNYA
    is_new_trip = False
    if i == 0:
        is_new_trip = True
    elif 'trip_id' in df_test.columns and df_test.at[i, 'trip_id'] != df_test.at[i-1, 'trip_id']:
        is_new_trip = True
        
    if not is_new_trip:
        # Ambil hasil prediksi detik sebelumnya (i-1)
        prev_pred_1 = predictions[i-1]
        
        # Ambil hasil prediksi 2 detik lalu (i-2), cek trip id yg sama
        if i > 1 and df_test.at[i, 'trip_id'] == df_test.at[i-2, 'trip_id']:
            prev_pred_2 = predictions[i-2]
        else:
            prev_pred_2 = prev_pred_1 # Fallback
            
        # --- UPDATE FITUR UTAMA DI DATAFRAME ---

        df_test.at[i, 'speed_mps_prev1'] = prev_pred_1
        df_test.at[i, 'speed_mps_prev2'] = prev_pred_2
        
    else:
        #awal trip, kita asumsikan start dari 0 atau data asli t=0
        df_test.at[i, 'speed_mps_prev1'] = 0.0
        df_test.at[i, 'speed_mps_prev2'] = 0.0

    # B. PREPARE INPUT VECTOR
    X_row = df_test.loc[i, feature_cols].values.reshape(1, -1)
    
    # C. SCALE
    X_row_scaled = scaler.transform(X_row)
    
    # D. PREDICT
    pred_speed = speed_model.predict(X_row_scaled)[0]
    
    if pred_speed < 0: 
        pred_speed = 0.0
        
    predictions.append(pred_speed)

# 4. Simpan ke DataFrame
df_test['predicted_speed'] = predictions
y_pred_speed = np.array(predictions)

# 5. Hitung Akselerasi berdasarkan PREDIKSI SPEED (Speed Difference Method)
# Accel = Pred_Speed(t) - Pred_Speed(t-1)
# Hati-hati: t-1 nya harus dari trip yang sama.
if 'trip_id' in df_test.columns:
    df_test['pred_speed_prev'] = df_test.groupby('trip_id')['predicted_speed'].shift(1).fillna(0)
    df_test['predicted_accel'] = df_test['predicted_speed'] - df_test['pred_speed_prev']
else:
    df_test['pred_speed_prev'] = df_test['predicted_speed'].shift(1).fillna(0)
    df_test['predicted_accel'] = df_test['predicted_speed'] - df_test['pred_speed_prev']

y_pred_accel = df_test['predicted_accel'].values
y_test_speed = df_test['speed_mps'].values
y_test_accel = df_test['acceleration'].values

print("✅ Autoregressive prediction complete.")
print(f"   Speed predictions shape: {y_pred_speed.shape}")
print(f"   Accel predictions shape: {y_pred_accel.shape}")

In [ ]:
# CELL 8: Calculate Metrics
print("\n=== Calculating Quality Metrics ===")

# Speed Metrics
speed_r2 = r2_score(y_test_speed, y_pred_speed)
speed_rmse = np.sqrt(mean_squared_error(y_test_speed, y_pred_speed))
speed_mae = mean_absolute_error(y_test_speed, y_pred_speed)
speed_mse = mean_squared_error(y_test_speed, y_pred_speed)

# Speed MAPE
y_speed_safe = np.where(y_test_speed == 0, 1e-6, y_test_speed)
speed_mape = np.mean(np.abs((y_test_speed - y_pred_speed) / y_speed_safe)) * 100

# Acceleration Metrics
accel_r2 = r2_score(y_test_accel, y_pred_accel)
accel_rmse = np.sqrt(mean_squared_error(y_test_accel, y_pred_accel))
accel_mae = mean_absolute_error(y_test_accel, y_pred_accel)
accel_mse = mean_squared_error(y_test_accel, y_pred_accel)

# Accel MAPE
y_accel_safe = np.where(y_test_accel == 0, 1e-6, y_test_accel)
accel_mape = np.mean(np.abs((y_test_accel - y_pred_accel) / y_accel_safe)) * 100

# Statistical Comparison
speed_mean_actual = np.mean(y_test_speed)
speed_mean_pred = np.mean(y_pred_speed)
speed_std_actual = np.std(y_test_speed)
speed_std_pred = np.std(y_pred_speed)

accel_mean_actual = np.mean(y_test_accel)
accel_mean_pred = np.mean(y_pred_accel)
accel_std_actual = np.std(y_test_accel)
accel_std_pred = np.std(y_pred_accel)

print("\n📊 SPEED PREDICTION METRICS:")
print("=" * 50)
print(f"R² Score:     {speed_r2:.4f}")
print(f"RMSE:         {speed_rmse:.4f} m/s ({speed_rmse * 3.6:.2f} km/h)")
print(f"MAE:          {speed_mae:.4f} m/s ({speed_mae * 3.6:.2f} km/h)")
print(f"MSE:          {speed_mse:.4f}")
print(f"MAPE:         {speed_mape:.2f}%")
print(f"Mean Actual:  {speed_mean_actual:.4f} m/s")
print(f"Mean Pred:    {speed_mean_pred:.4f} m/s")
print(f"Std Actual:   {speed_std_actual:.4f} m/s")
print(f"Std Pred:     {speed_std_pred:.4f} m/s")

print("\n📊 ACCELERATION PREDICTION METRICS (from speed diff):")
print("=" * 50)
print(f"R² Score:     {accel_r2:.4f}")
print(f"RMSE:         {accel_rmse:.4f} m/s²")
print(f"MAE:          {accel_mae:.4f} m/s²")
print(f"MSE:          {accel_mse:.4f}")
print(f"MAPE:         {accel_mape:.2f}%")
print(f"Mean Actual:  {accel_mean_actual:.4f} m/s²")
print(f"Mean Pred:    {accel_mean_pred:.4f} m/s²")
print(f"Std Actual:   {accel_std_actual:.4f} m/s²")
print(f"Std Pred:     {accel_std_pred:.4f} m/s²")

# DEBUG: Detailed acceleration analysis
print(f"\n🔍 DEBUG: Why Is Acceleration Bad?")
print(f"   Mean difference: {accel_mean_pred - accel_mean_actual:.4f} m/s² (should be ~0)")
print(f"   Std ratio: {accel_std_pred / accel_std_actual:.4f} (should be ~1)")
print(f"   Bias: {np.mean(y_pred_accel - y_test_accel):.4f} m/s²")

# Check if speed predictions are smooth (causing accel issues)
speed_changes = np.diff(y_pred_speed)
actual_speed_changes = np.diff(y_test_speed)
print(f"\n   Speed change analysis:")
print(f"     Predicted speed std of changes: {np.std(speed_changes):.4f}")
print(f"     Actual speed std of changes: {np.std(actual_speed_changes):.4f}")
print(f"     Ratio: {np.std(speed_changes) / np.std(actual_speed_changes):.4f}")

if accel_r2 < 0:
    print(f"\n⚠️  NEGATIVE R²! This means:")
    print(f"     - Model is worse than predicting mean")
    print(f"     - Likely causes:")
    print(f"       1. Speed predictions too smooth (low variance)")
    print(f"       2. Model not capturing acceleration dynamics")
    print(f"       3. Test set very different from training")
    print(f"       4. Model predicting absolute speed, not changes")

In [ ]:
# CELL 9: Create Visualization
print("\nGenerating validation plots...")

# Create two figure sets
# Figure 1: Statistical Plots (6 subplots)
fig1 = plt.figure(figsize=(18, 12))

# Plot 1: Speed - Actual vs Predicted Scatter
plt.subplot(3, 3, 1)
plt.scatter(y_test_speed, y_pred_speed, alpha=0.3, s=1)
plt.plot([y_test_speed.min(), y_test_speed.max()], 
         [y_test_speed.min(), y_test_speed.max()], 
         'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Speed (m/s)')
plt.ylabel('Predicted Speed (m/s)')
plt.title(f'Speed: Actual vs Predicted\nR²={speed_r2:.4f}, RMSE={speed_rmse:.4f} m/s')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Acceleration - Actual vs Predicted Scatter
plt.subplot(3, 3, 2)
plt.scatter(y_test_accel, y_pred_accel, alpha=0.3, s=1)
plt.plot([y_test_accel.min(), y_test_accel.max()], 
         [y_test_accel.min(), y_test_accel.max()], 
         'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Acceleration (m/s²)')
plt.ylabel('Predicted Acceleration (m/s²)')
plt.title(f'Accel: Actual vs Predicted (from speed diff)\nR²={accel_r2:.4f}, RMSE={accel_rmse:.4f} m/s²')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Speed Error Distribution
plt.subplot(3, 3, 3)
speed_errors = y_test_speed - y_pred_speed
plt.hist(speed_errors, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
plt.xlabel('Prediction Error (m/s)')
plt.ylabel('Frequency')
plt.title(f'Speed Error Distribution\nMAE={speed_mae:.4f} m/s')
plt.grid(True, alpha=0.3)

# Plot 4: Acceleration Error Distribution
plt.subplot(3, 3, 4)
accel_errors = y_test_accel - y_pred_accel
plt.hist(accel_errors, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
plt.xlabel('Prediction Error (m/s²)')
plt.ylabel('Frequency')
plt.title(f'Acceleration Error Distribution\nMAE={accel_mae:.4f} m/s²')
plt.grid(True, alpha=0.3)

# Plot 5: Speed Statistics Comparison
plt.subplot(3, 3, 5)
categories = ['Mean', 'Std Dev']
actual_vals = [speed_mean_actual, speed_std_actual]
pred_vals = [speed_mean_pred, speed_std_pred]
x = np.arange(len(categories))
width = 0.35
plt.bar(x - width/2, actual_vals, width, label='Actual', color='blue', alpha=0.7)
plt.bar(x + width/2, pred_vals, width, label='Predicted', color='red', alpha=0.7)
plt.ylabel('Speed (m/s)')
plt.title('Speed Statistics Comparison')
plt.xticks(x, categories)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 6: Acceleration Statistics Comparison
plt.subplot(3, 3, 6)
actual_vals = [accel_mean_actual, accel_std_actual]
pred_vals = [accel_mean_pred, accel_std_pred]
plt.bar(x - width/2, actual_vals, width, label='Actual', color='blue', alpha=0.7)
plt.bar(x + width/2, pred_vals, width, label='Predicted', color='red', alpha=0.7)
plt.ylabel('Acceleration (m/s²)')
plt.title('Acceleration Statistics Comparison')
plt.xticks(x, categories)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 7: Speed Time Series (Sample)
plt.subplot(3, 3, 7)
sample_size = min(500, len(y_test_speed))
sample_idx = np.linspace(0, len(y_test_speed)-1, sample_size).astype(int)
plt.plot(sample_idx, y_test_speed[sample_idx], 'b-', alpha=0.7, label='Actual', linewidth=1)
plt.plot(sample_idx, y_pred_speed[sample_idx], 'r-', alpha=0.7, label='Predicted', linewidth=1)
plt.xlabel('Sample Index')
plt.ylabel('Speed (m/s)')
plt.title('Speed: Prediction vs Actual (Time Series)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 8: Acceleration Time Series (Sample)
plt.subplot(3, 3, 8)
plt.plot(sample_idx, y_test_accel[sample_idx], 'b-', alpha=0.7, label='Actual', linewidth=1)
plt.plot(sample_idx, y_pred_accel[sample_idx], 'r-', alpha=0.7, label='Predicted', linewidth=1)
plt.xlabel('Sample Index')
plt.ylabel('Acceleration (m/s²)')
plt.title('Accel: Prediction vs Actual (Time Series)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 9: Speed Residuals Over Time
plt.subplot(3, 3, 9)
plt.plot(sample_idx, speed_errors[sample_idx], 'g-', alpha=0.5, linewidth=0.8)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.axhline(y=speed_mae, color='orange', linestyle=':', linewidth=1, label=f'MAE={speed_mae:.4f}')
plt.axhline(y=-speed_mae, color='orange', linestyle=':', linewidth=1)
plt.xlabel('Sample Index')
plt.ylabel('Error (m/s)')
plt.title('Speed Prediction Errors Over Time')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
print("✅ Statistical plots generated")

In [ ]:
# CELL 10: Evaluate Pass/Fail & Save Results
print("\n=== QUALITY EVALUATION ===")

# Compile metrics
metrics = {
    "model_name": model_name,
    "run_timestamp": RUN_TIMESTAMP,
    "test_samples": int(len(y_test_speed)),
    "acceleration_method": "speed_difference",  # YOUR METHOD!
    "speed": {
        "r2_score": float(speed_r2),
        "rmse_ms": float(speed_rmse),
        "rmse_kmh": float(speed_rmse * 3.6),
        "mae_ms": float(speed_mae),
        "mae_kmh": float(speed_mae * 3.6),
        "mse": float(speed_mse),
        "mape_percent": float(speed_mape),
        "mean_actual": float(speed_mean_actual),
        "mean_predicted": float(speed_mean_pred),
        "std_actual": float(speed_std_actual),
        "std_predicted": float(speed_std_pred)
    },
    "acceleration": {
        "r2_score": float(accel_r2),
        "rmse": float(accel_rmse),
        "mae": float(accel_mae),
        "mse": float(accel_mse),
        "mape_percent": float(accel_mape),
        "mean_actual": float(accel_mean_actual),
        "mean_predicted": float(accel_mean_pred),
        "std_actual": float(accel_std_actual),
        "std_predicted": float(accel_std_pred)
    },
    "thresholds": {
        "min_r2_score": MIN_R2_SCORE,
        "max_speed_rmse_ms": MAX_SPEED_RMSE,
        "max_accel_rmse": MAX_ACCEL_RMSE,
        "max_speed_mae_ms": MAX_SPEED_MAE,
        "max_accel_mae": MAX_ACCEL_MAE
    }
}

# Determine pass/fail
failures = []
if speed_r2 < MIN_R2_SCORE:
    failures.append(f"Speed R² {speed_r2:.4f} < {MIN_R2_SCORE}")
if speed_rmse > MAX_SPEED_RMSE:
    failures.append(f"Speed RMSE {speed_rmse:.4f} > {MAX_SPEED_RMSE} m/s")
if speed_mae > MAX_SPEED_MAE:
    failures.append(f"Speed MAE {speed_mae:.4f} > {MAX_SPEED_MAE} m/s")
if accel_rmse > MAX_ACCEL_RMSE:
    failures.append(f"Acceleration RMSE {accel_rmse:.4f} > {MAX_ACCEL_RMSE} m/s²")
if accel_mae > MAX_ACCEL_MAE:
    failures.append(f"Acceleration MAE {accel_mae:.4f} > {MAX_ACCEL_MAE} m/s²")

metrics["validation_passed"] = len(failures) == 0
metrics["failures"] = failures

# Print results
print(json.dumps(metrics, indent=2))

# Save plot
img_buf = io.BytesIO()
fig1.savefig(img_buf, format='png', dpi=150, bbox_inches='tight')
img_buf.seek(0)
with fs.open(OUTPUT_PLOT_PATH, 'wb') as f:
    f.write(img_buf.getbuffer())
print(f"\n✅ Plots saved to {OUTPUT_PLOT_PATH}")

# Save metrics JSON
with fs.open(OUTPUT_METRICS_PATH, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"✅ Metrics saved to {OUTPUT_METRICS_PATH}")

# Final verdict
if failures:
    print(f"\n⚠️  Model Quality Validation Failed ({len(failures)} issues):")
    for failure in failures:
        print(f"  - {failure}")
    print(f"\n💡 Note: Thresholds have been relaxed for limited data testing")
else:
    print("\n✅ Model Quality Validation Passed!")
    print("   All metrics within acceptable thresholds.")

print("\n=== Validation Complete ===")